# Control Test

## 摄像头功能测试

In [5]:
import cv2
import ipywidgets.widgets as widgets
import threading
import time

In [6]:
import enum

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])   

In [7]:
# 打开摄像头
image = cv2.VideoCapture(0)

# width=1280
# height=960
# cap.set(cv2.CAP_PROP_FRAME_WIDTH,width)   # 设置图像宽度
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT,height) # 设置图像高度


image.set(3,600)       
image.set(4,500)
image.set(5, 30)  # 设置帧率

image.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
image.set(cv2.CAP_PROP_BRIGHTNESS, 40) # 设置亮度 -64 - 64  0.0
image.set(cv2.CAP_PROP_CONTRAST, 50)   # 设置对比度 -64 - 64  2.0
image.set(cv2.CAP_PROP_EXPOSURE, 156)  # 设置曝光值 1.0 - 5000  156.0


[ WARN:0] global /tmp/pip-req-build-xw6jtoah/opencv/modules/videoio/src/cap_v4l.cpp (890) open VIDEOIO(V4L2:/dev/video0): can't open camera by index


False

In [8]:
# 设置摄像头显示组件
image_widget = widgets.Image(format='jpeg', width=500, height=400) 
# 显示摄像头组件
display(image_widget)

TypeError: 'module' object is not callable

In [ ]:
while 1:
    ret, frame = image.read()
    image_widget.value = bgr8_to_jpeg(frame)
    time.sleep(0.010)

In [ ]:
image.release()

## 摄像机云台转动测试

In [ ]:
import RPi.GPIO as GPIO
import time

In [ ]:
# 舵机引脚定义
'''
    摄像头云台左右舵机是插在扩展板的S2 -> 11
    摄像头云台上下舵机插在扩展板上的S3 -> 9
'''

left_N_right_Pin= 11
up_N_down_Pin = 9

# 设置GPIO口为BCM编码方式

GPIO.setmode(GPIO.BCM)

# 忽略警告信息

GPIO.setwarnings(False)

In [ ]:
# 舵机引脚设置为输出模式
def init(Pin):
    GPIO.setup(Pin, GPIO.OUT)

# 定义一个脉冲函数，用来模拟方式产生pwm值. 
# 时基脉冲为20ms，该脉冲高电平部分在0.5-2.5ms控制0-180度
def servo_pulse(Pin, angle):
    pulsewidth = (angle * 11) + 500
    GPIO.output(Pin, GPIO.HIGH)
    time.sleep(pulsewidth/1000000.0)
    GPIO.output(Pin, GPIO.LOW)
    time.sleep(20.0/1000-pulsewidth/1000000.0)

In [ ]:
# 左右转动测试

init(left_N_right_Pin)

for pos in range(20, 161):
    servo_pulse(left_N_right_Pin, pos)
    time.sleep(0.009)

for pos in reversed(range(20, 161)):
    servo_pulse(left_N_right_Pin, pos)
    time.sleep(0.009)

time.sleep(2)

In [ ]:
# 上下转动测试

init(up_N_down_Pin)

for pos in range(80, 161):
    servo_pulse(up_N_down_Pin, pos)
    time.sleep(0.009)

for pos in reversed(range(80, 161)):
    servo_pulse(up_N_down_Pin, pos)
    time.sleep(0.009)

time.sleep(2)

## 小车运动控制

In [ ]:
import RPi.GPIO as GPIO
import time

In [ ]:
# 小车电机引脚定义
IN1 = 20
IN2 = 21
IN3 = 19
IN4 = 26
ENA = 16
ENB = 13

# 设置GPIO口为BCM编码方式
GPIO.setmode(GPIO.BCM)

In [ ]:
# 电机引脚初始化操作

def motor_init():
    global pwm_ENA
    global pwm_ENB
    global delaytime

    GPIO.setup(ENA,GPIO.OUT,initial=GPIO.HIGH)
    GPIO.setup(IN1,GPIO.OUT,initial=GPIO.LOW)
    GPIO.setup(IN2,GPIO.OUT,initial=GPIO.LOW)
    GPIO.setup(ENB,GPIO.OUT,initial=GPIO.HIGH)
    GPIO.setup(IN3,GPIO.OUT,initial=GPIO.LOW)
    GPIO.setup(IN4,GPIO.OUT,initial=GPIO.LOW)

    #设置pwm引脚和频率为2000hz
    pwm_ENA = GPIO.PWM(ENA, 2000)
    pwm_ENB = GPIO.PWM(ENB, 2000)
    pwm_ENA.start(0)
    pwm_ENB.start(0)

In [ ]:
# 小车前进
def run(delaytime):
    GPIO.output(IN1, GPIO.HIGH)
    GPIO.output(IN2, GPIO.LOW)
    GPIO.output(IN3, GPIO.HIGH)
    GPIO.output(IN4, GPIO.LOW)

    pwm_ENA.ChangeDutyCycle(80)
    pwm_ENB.ChangeDutyCycle(80)
    time.sleep(delaytime)

# 小车后退
def back(delaytime):
    GPIO.output(IN1, GPIO.LOW)
    GPIO.output(IN2, GPIO.HIGH)
    GPIO.output(IN3, GPIO.LOW)
    GPIO.output(IN4, GPIO.HIGH)

    pwm_ENA.ChangeDutyCycle(80)
    pwm_ENB.ChangeDutyCycle(80)
    time.sleep(delaytime)

# 小车左转
def left(delaytime):
    GPIO.output(IN1, GPIO.LOW)
    GPIO.output(IN2, GPIO.LOW)
    GPIO.output(IN3, GPIO.HIGH)
    GPIO.output(IN4, GPIO.LOW)

    pwm_ENA.ChangeDutyCycle(80)
    pwm_ENB.ChangeDutyCycle(80)
    time.sleep(delaytime)

# 小车右转
def right(delaytime):
    GPIO.output(IN1, GPIO.HIGH)
    GPIO.output(IN2, GPIO.LOW)
    GPIO.output(IN3, GPIO.LOW)
    GPIO.output(IN4, GPIO.LOW)

    pwm_ENA.ChangeDutyCycle(80)
    pwm_ENB.ChangeDutyCycle(80)
    time.sleep(delaytime)

# 小车原地左转
def spin_left(delaytime):
    GPIO.output(IN1, GPIO.LOW)
    GPIO.output(IN2, GPIO.HIGH)
    GPIO.output(IN3, GPIO.HIGH)
    GPIO.output(IN4, GPIO.LOW)

    pwm_ENA.ChangeDutyCycle(80)
    pwm_ENB.ChangeDutyCycle(80)
    time.sleep(delaytime)

# 小车原地右转
def spin_right(delaytime):
    GPIO.output(IN1, GPIO.HIGH)
    GPIO.output(IN2, GPIO.LOW)
    GPIO.output(IN3, GPIO.LOW)
    GPIO.output(IN4, GPIO.HIGH)

    pwm_ENA.ChangeDutyCycle(80)
    pwm_ENB.ChangeDutyCycle(80)
    time.sleep(delaytime)

# 小车停止
def brake(delaytime):
    GPIO.output(IN1, GPIO.LOW)
    GPIO.output(IN2, GPIO.LOW)
    GPIO.output(IN3, GPIO.LOW)
    GPIO.output(IN4, GPIO.LOW)

    pwm_ENA.ChangeDutyCycle(80)
    pwm_ENB.ChangeDutyCycle(80)
    time.sleep(delaytime)

In [ ]:
motor_init()

In [ ]:
# 运动控制
run(1)
brake(1)

left(1)
brake(1)

right(1)
brake(1)

back(1)
brake(1)